In [1]:
import torch
import transformers

from life_after_bert import MCDataset, evaluate_decoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "gpt2-large"
task_name = "Age Comparison"
num_choices = 2

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, mask_token="[MASK]")  # Causal LM's don't have mask tokens by default
dataset = MCDataset.load_data(task_name, num_choices, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2022-04-21 11:42:54 | INFO | data.py | Loading jsonl file from /home/kzhao/life-after-bert/tests/data/oLMpics_age_comparison_dev.jsonl


Using pad_token, but it is not set yet.


2022-04-21 11:42:55 | WARNING | data.py | Setting tokenizer's pad_token to eos_token


In [4]:
accuracy, (all_answers, all_preds) = evaluate_decoder(model, tokenizer, dataset, device=device)
accuracy

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

0.696